***
### Packages

In [ ]:
# Packages
import math, re, itertools
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
# -------------------------------
from tenpy.networks.mps import MPS
from tenpy.networks.site import SpinHalfSite
from qiskit.quantum_info import *

***
# Shadow Tomography + MPS State Tomography
***

## Test State
To test I will be using the $\ket{w}$ state but I will be writing the code in a general way which in theory would allow any state to be run though this notebook. The state can also be redefined at any place in the notebook but as of now this is what will be used.

# MPS
## Generate the MPS matrices 

The maximum bond dimension for a system of $n$ qubits is $2^{n/2}$. 

From the PennyLane demo which can be found at: https://pennylane.ai/qml/demos/tutorial_mps. 

> Checked by hand and currently this method is not working.

In [638]:
# |w> state vector.
# state = Statevector.from_label("0001") + Statevector.from_label("0010") + Statevector.from_label("0100") + Statevector.from_label("1000")
state = Statevector.from_label("00000") + Statevector.from_label("11111")
state = state/np.sqrt(state.inner(state))

state.draw("latex")

<IPython.core.display.Latex object>

In [639]:
# From PennyLane Demo on MPS.
def split(M, bond_dim):
    """Split a matrix M via SVD and keep only the ``bond_dim`` largest entries."""
    U, S, Vd = np.linalg.svd(M, full_matrices=False)
    bonds = len(S)
    Vd = Vd.reshape(bonds, 2, -1)
    U = U.reshape((-1, 2, bonds))

    # keep only chi bonds
    chi = np.min([bonds, bond_dim])
    U, S, Vd = U[:, :, :chi], S[:chi], Vd[:chi]
    return U, S, Vd

def mpsDecomp(psi, bond_dim):
    """Turn a state vector ``psi`` into an MPS with bond dimension ``bond_dim``."""
    Ms = []
    Ss = []

    n = psi.num_qubits

    psi = np.reshape(psi, (2, -1))   # split psi[2, 2, 2, 2..] = psi[2, (2x2x2...)]
    U, S, Vd = split(psi, bond_dim)  # psi[2, (2x2x..)] = U[2, mu] S[mu] Vd[mu, (2x2x2x..)]

    Ms.append(np.reshape(U, (2, 1, 2)))
    Ss.append(S)
    bondL = Vd.shape[0]
    psi = np.tensordot(np.diag(S), Vd, 1)

    for _ in range(n-2):
        psi = np.reshape(psi, (2*bondL, -1)) # reshape psi[2 * bondL, (2x2x2...)]
        U, S, Vd = split(psi, bond_dim) # psi[2, (2x2x..)] = U[2, mu] S[mu] Vd[mu, (2x2x2x..)]
        Ms.append(U)
        Ss.append(S)

        psi = np.tensordot(np.diag(S), Vd, 1)
        bondL = Vd.shape[0]

    # dummy step on last site
    psi = np.reshape(psi, (-1, 1))
    U, _, _ = np.linalg.svd(psi, full_matrices=False)

    U = np.reshape(U, (-1, 2, 1))
    Ms.append(U)

    Ls = []
    for i, U in enumerate(Ms):
        if i == 0:
            Ls.append(U)
        else:
            L = np.linalg.pinv(np.diag(Ss[i-1])) @ Ms[i]
            Ls.append(L)
    
    return (Ls, Ss, Ms)


# Currently decomissioned. Might fix later.
# def mpsDecomp(state, bond_dim): # Currently fixed bond_dim to 2 and outputs Us (singular values are included in Us). To get the general gamma lambda form, just divide the singular values from the Us and return them separately.
#     # State information
#     n = state.num_qubits
#     psi = state.data

#     Us = []
#     Ss = []

#     psi = np.reshape(psi, tuple(2 for _ in range(n)))

#     for i in range(n-1):
#         # Reshape it into a matrix. This is effively partitioning the state vector into two parts. The first part is the first qubit, and the second part is the rest of the qubits.
#         psi = np.reshape(psi, (2,-1))
#         U, S, V = np.linalg.svd(psi, full_matrices=False)

#         print(psi)
#         print(psi.shape)
#         print(U, S, V)

#         Ss.append(S)
#         if i == 0:
#             Us.append(np.reshape(U, (2, 1, 2)))
#         elif i == n-2:
#             Us.append(np.reshape(U, (2, 2, 2)))
#             V = np.tensordot(np.diag(S), V, 1)
#             Us.append(np.reshape(V, (2, 2, 1)))
#         else:
#             Us.append(np.reshape(U, (2, 2, 2)))

#         psi = np.reshape(psi, tuple(2 for _ in range(n-i-1)))
#         psi = np.tensordot(np.diag(S), V, 2)
#         print(psi)

#     Ls = []
#     for i, U in enumerate(Us):
#         if i == 0:
#             Ls.append(U)
#         else:
#             L = np.linalg.pinv(np.diag(Ss[i-1])) @ Us[i]
#             Ls.append(L)
    
#     return (Ls, Ss, Us)

## Test

In [640]:
mpsState = mpsDecomp(state, 2)
print(f'{mpsState[0]} \n\n {mpsState[1]} \n\n {mpsState[2]}')

[array([[[1.+0.j, 0.+0.j]],

       [[0.+0.j, 1.+0.j]]]), array([[[1.41421356+0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j]],

       [[0.        +0.j, 0.        +0.j],
        [0.        +0.j, 1.41421356+0.j]]]), array([[[1.41421356+0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j]],

       [[0.        +0.j, 0.        +0.j],
        [0.        +0.j, 1.41421356+0.j]]]), array([[[ 1.41421356+0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j]],

       [[ 0.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -1.41421356+0.j]]]), array([[[ 1.+0.j],
        [ 0.+0.j]],

       [[ 0.+0.j],
        [-1.+0.j]]])] 

 [array([0.70710678, 0.70710678]), array([0.70710678, 0.70710678]), array([0.70710678, 0.70710678]), array([0.70710678, 0.70710678])] 

 [array([[[1.+0.j, 0.+0.j]],

       [[0.+0.j, 1.+0.j]]]), array([[[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]],

       [[0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]]), array([[[1.+0.j, 0.+0.j],
  


## Sanity Test: Reconstruct State from MPS Matrices

In [641]:
def mps_to_state_vec(Ls, Ss):

    state = Statevector(data=np.zeros(2**len(Ls)))
    for p in itertools.product(range(2), repeat=len(Ls)):
        for n,i in enumerate(p):
            if n == 0:
               c = Ls[0][i]
            else:
                c = c @ np.diag(Ss[n-1]) @ Ls[n][i]

        state += Statevector.from_label("".join(map(str, p)))*c[0][0]
        
    return state


mps_to_state_vec(mpsState[0], mpsState[1]).draw("latex")

<IPython.core.display.Latex object>

## Expectation value from MPS

https://numpy.org/doc/2.1/reference/generated/numpy.einsum.html

https://arxiv.org/pdf/1008.3477


In [642]:
def mpsSingleSiteExpectation(Ls, Ss, Observable, Site):
    """Calculate the expectation value of a single-site observable for an MPS."""

    if Observable == "X":
        O = np.array([[0, 1], [1, 0]])
    elif Observable == "Y":
        O = np.array([[0, -1j], [1j, 0]])
    elif Observable == "Z":
        O = np.array([[1, 0], [0, -1]])
    elif Observable == "I":
        O = np.array([[1, 0], [0, 1]])

    L = Ls[Site-1]
    if Site == 1:
        A = np.einsum('ijk, jk -> ijk', L, np.diag(Ss[Site-1]))
        Astar = A.conj()
    elif Site == len(Ls):
        A = np.einsum('jk, ijk -> ijk', np.diag(Ss[Site-2]), L)
        Astar = A.conj()
    else:
        A = np.einsum('db, abc -> adc', np.diag(Ss[Site-2]), L )
        A = np.einsum('abc, cd -> abd', A, np.diag(Ss[Site-1]))
        Astar = A.conj()

    res = np.einsum('abc, da -> dbc', Astar, O)
    return np.einsum('abc, abc ->', res, A)

for i in ["X", "Y", "Z", "I"]:
    for j in range(1, 4):
        print(f"Expectation value of {i} on site {j}: {mpsSingleSiteExpectation(mpsState[0], mpsState[1], i, j)}")
        print(f"Theoretical value: {state.expectation_value(Pauli(i), [j-1])}")
        print("\n")

def mpsNeighbourSiteExpectation(A, ObservableArray):
    
    O = [1]
    for Observable in ObservableArray:
        if Observable == "X":
            O = np.kron(O, np.array([[0, 1], [1, 0]]))
        elif Observable == "Y":
            O = np.kron(O, np.array([[0, -1j], [1j, 0]]))
        elif Observable == "Z":
            O = np.kron(O, np.array([[1, 0], [0, -1]]))
        elif Observable == "I":
            O = np.kron(O, np.array([[1, 0], [0, 1]]))

    O = np.reshape(O, [2,2,2,2])

    Astar = A.conj()
    res = np.einsum('abcd, abef -> efcd', Astar, O)
    return(np.einsum('abcd, abcd ->', res, A))

Ls = mpsState[0]
Ss = mpsState[1]
Us = mpsState[2]

A = np.einsum('db, abc -> adc', np.diag([1,1]), Ls[0])
A = np.einsum('abc, cd -> abd', A, np.diag(Ss[0]))
A = np.einsum('abc, dcf -> adbf', A, Ls[1])
A = np.einsum('abcd, de -> abce', A, np.diag(Ss[1]))
for pair in itertools.combinations_with_replacement(["X", "Y", "Z", "I"], 2):
    print(f"Expectation value of {''.join(pair)} between sites 1 and 2: {mpsNeighbourSiteExpectation(A, pair)/2}")
    print(f"Theoretical value: {state.expectation_value(Pauli(''.join(pair)))}")
    print("\n")
    


Expectation value of X on site 1: 0j
Theoretical value: 0.0


Expectation value of X on site 2: 0j
Theoretical value: 0.0


Expectation value of X on site 3: 0j
Theoretical value: 0.0


Expectation value of Y on site 1: 0j
Theoretical value: 0.0


Expectation value of Y on site 2: 0j
Theoretical value: 0.0


Expectation value of Y on site 3: 0j
Theoretical value: 0.0


Expectation value of Z on site 1: 0j
Theoretical value: 0.0


Expectation value of Z on site 2: 0j
Theoretical value: 0.0


Expectation value of Z on site 3: 0j
Theoretical value: 0.0


Expectation value of I on site 1: (0.9999999999999998+0j)
Theoretical value: 0.9999999999999999


Expectation value of I on site 2: (0.9999999999999998+0j)
Theoretical value: 0.9999999999999999


Expectation value of I on site 3: (0.9999999999999998+0j)
Theoretical value: 0.9999999999999999


Expectation value of XX between sites 1 and 2: 0j
Theoretical value: 0.0


Expectation value of XY between sites 1 and 2: 0j
Theoretical value: 0.0


In [643]:
print(state.expectation_value(Pauli('ZZ')))

Ls = mpsState[0]
Ss = mpsState[1]
Us = mpsState[2]

Site = 2
A = Ls[Site - 1]
A = np.einsum('abc, cd -> abd',  A, np.diag(Ss[Site-2]))
A = np.einsum('abc, bd -> adc', A, np.diag(Ss[Site-1]))

Astar = A.conj()
print(np.einsum('ijk,ijk', Astar, A))

O = np.array([[0, 1], [1, 0]])
print(A)
print('\n\n')
res = np.einsum('abc, da -> dbc', Astar, O)
print(res)
print(np.einsum('abc, abc ->', res, A))

T = [[2, 0], 
     [0, 2]]

P = [[0, 1], 
     [1, 0]]

print(np.einsum('ab, bc -> ac', T, P))

0.9999999999999998
(0.9999999999999998+0j)
[[[0.70710678+0.j 0.        +0.j]
  [0.        +0.j 0.        +0.j]]

 [[0.        +0.j 0.        +0.j]
  [0.        +0.j 0.70710678+0.j]]]



[[[0.        +0.j 0.        +0.j]
  [0.        +0.j 0.70710678+0.j]]

 [[0.70710678+0.j 0.        +0.j]
  [0.        +0.j 0.        +0.j]]]
0j
[[0 2]
 [2 0]]


# Simulating Measurements
Testing how the qiskit can simulate quantum measurements of observables.

In [644]:
#Transform the state vector so measuremring in the Z basis is equivalent to measuring in the X basis as well as the Y basis.
n = state.num_qubits
statez = state
statey = state.evolve(Operator.from_label("S"*n).conjugate()).evolve(Operator.from_label("H"*n))
statex = state.evolve(Operator.from_label("H"*n))

print(statex.measure()[0])
print(statey.measure()[0])
print(statez.measure()[0])

00101
01011
00000


# Shadow Tomography
These were ported from my past work but using a different package to model quantum states has introduced some errors that need to be fixed. Also these methods are not optimized for the new package.

In [645]:
def generate_pauli_measurement_schemes(num_qubits, size):
    """
    Description:
        Generate a list of Pauli measurement schemes for a given number of qubits. Each measurement scheme is a list of Pauli measurements to be performed on each qubit.
    
    Args:
        num_qubits (int): The number of qubits.
        size (int): The number of measurement schemes to generate.

    Returns:
        measurement_schemes (list): A list of measurement schemes. Each measurement scheme is a list of Pauli measurements to be performed on each
        qubit. The Pauli measurements are represented by the strings 'X', 'Y', and 'Z'.
    """
    # Generate the measurement schemes.
    measurement_schemes = np.random.choice(['X', 'Y', 'Z'], (size, num_qubits))

    # Write the measurement schemes to a file.
    return np.array(measurement_schemes)

In [646]:
def generate_classical_shadow(state, measurement_schemes):
    """
    Description:
        Given a quantum state and a set of measurement schemes, generate the classical shadow of the state.
        Adapted from https://pennylane.ai/qml/demos/tutorial_classical_shadows/.

    Args:
        state (array): The quantum state.
        measurement_schemes (array): The measurement schemes.

    Returns:
        measurements (array): The classical shadows of the state.
    """
    # Get the measurements schemes from the file.
    size, num_qubits = measurement_schemes.shape

    # Create a matrix to store the results of the measurements.
    measurements = []

    # Perform the measurements.
    for m in measurement_schemes:
        t1 = ""
        t2 = ""
        for p in m:
            if p == 'X':
                t1 += "I"
                t2 += "H"
            elif p == 'Y':
                t1 += "S"
                t2 += "H"
            else:
                t1 += "I"
                t2 += "I"

        # Perform the measurements.
        rotatedStat = state.evolve(Operator.from_label(t1).conjugate()).evolve(Operator.from_label(t2))
        measurements.append(list(rotatedStat.measure()[0]))
        
    return np.array(measurements)

In [647]:
def estimate_pauli_observable(measurements, measurement_schemes, observable, k = 10):
    """
    Adapted from https://pennylane.ai/qml/demos/tutorial_classical_shadows/ and the original paper by Preskill.

    Given a classical shadow, a Pauli observable and value k, this function estimates the expectation value of the observable using median of means on k estimators.


    """
    # Get the shadow size and the number of qubits.
    shadow_size = len(measurements)
    num_qubits = len(measurements[0])
    
    # Check if the observable and the number of qubits in the shadow match. That is, the observable is a tensor product of Pauli operators acting on the same number of qubits as the shadow.
    if len(observable) != num_qubits:
        raise ValueError("The observable and the number of qubits in the shadow do not match.")

    # Edit the observable to do pattern matching with the measurement string. Replacing 'I' with the single character wild card '.'.
    observable = observable.replace('I', '.')

    # Split the shadow into k subsets.
    measurements_splits = np.array_split(measurements, k)
    measurement_schemes_splits = np.array_split(measurement_schemes, k)
    shadows = list(zip(measurements_splits, measurement_schemes_splits))

    # Array to store the estimated expectation values from each estimator.
    means = np.zeros(k)

    # Iterate over the number of estimators. Calculate the expectation value of the observable for each estimator and store the result in the means array.
    for s, shadow in enumerate(shadows):
        
        # Unpack the shadow k.
        measurements_k, measurement_schemes_k = shadow

        # Count the number of matches between the observable and the measurement schemes.
        match_indices = []
        for i, measurement_scheme_k in enumerate(measurement_schemes_k):

            # Check if the measurement scheme matches the observable.
            if re.match(observable, ''.join(measurement_scheme_k)):
                match_indices.append(i)

        # Catch the case that there are no matches.
        if len(match_indices) > 0:
            # Sum over the matches.
            sum = 0
            for match in match_indices:
                # Calculate the sign of the expectation for the single shot measurement.
                sign = 1
                for k, measurement_k in enumerate(measurements_k[match]):
                    if observable[k] in ['X', 'Z']:
                        x = 1 if int(measurement_k) == 0 else -1
                        sign *= x
                    elif observable[k] == 'Y':
                        x = 1 if int(measurement_k) == 0 else -1
                        sign *= (-1 * x)

                sum += sign*3**(len(list(filter(lambda x: x != '.', observable))))
        else:
            sum = 0
            
        # Store the result in the means array.
        means[s] = sum/len(measurement_schemes_k)


    # Return the median of the means.
    return np.median(means)

## Sanity test: Estimate some simple observables 

In [648]:
# print(state.expectation_value(Operator.from_label("ZII")))
# print(state.expectation_value(Operator.from_label("ZIZ")))
# print(state.expectation_value(Operator.from_label("ZZZ")))

In [649]:
# Generate the measurement schemes.
n = state.num_qubits
measurement_schemes = generate_pauli_measurement_schemes(n, 5000)

# Generate the classical shadow.
# measurements = generate_classical_shadow(state, measurement_schemes)

# # Estimate the expectation value of the observable.
# observable = 'ZII'
# print(estimate_pauli_observable(measurements, measurement_schemes, observable, k = 50))
# # Estimate the expectation value of the observable.
# observable = 'ZIZ'
# print(estimate_pauli_observable(measurements, measurement_schemes, observable, k = 50))
# # Estimate the expectation value of the observable.
# observable = 'ZZZ'
# print(estimate_pauli_observable(measurements, measurement_schemes, observable, k = 50))

# MPS Tomography
Current thought is to randomly generate the tensor for each site along with singular values, flatten it and append the singular values then optimize that list of values over the simple function which is the average distance between the resulting expectation value and the true/measured expectation value. Have to reason how to deal with singular values effecting neighbouring site. Perhaps optimize two sites at a time.

In [650]:
def distance(tensorComps, shape, observables, expectations):
    """
    Description:    
        Calculate the distance between the estimated expectations and target expectations
    Args:
        tensorComps (array): The flattened tensor components.
        observables (list): A list of observables to optimize over.
        expectations (list): A list of the target expectations.
    Returns:
        distance (float): The distance between the estimated expectations and target
        expectations.
    """
    
    # Estimate expectation values using the reconstructed tensor.
    estimatedExpectations = []
    for o in observables:
        exp = mpsNeighbourSiteExpectation(np.reshape(tensorComps, shape), o)
        estimatedExpectations.append(exp)

    # Calculate the distance between the estimated expectations and target expectations.
    return np.linalg.norm(np.array(expectations) - np.array(estimatedExpectations))

def optimizeSite(A, observables, expectations):
    """
    Description:
        Given neighbouring sites M1 and M2, and the singular values S, optimize the tensor to match the target expectation.
    Args:
        M1 (array): The left tensor.
        M2 (array): The right tensor.
        S (array): The singular values.
        site (int): The site to optimize.
        observable (str): The observables to optimize over.
        expectation (float): The target expectations.
    Returns:
        M1 (array): The optimized left tensor.
        M2 (array): The optimized right tensor.
        S (array): The optimized singular values.
    """

    tenShape = A.shape

    res = minimize(distance, A.flatten(), args=(tenShape, observables, expectations))

    print(res.success)
    print(res.message)
    
    A = np.reshape(res.x, tenShape)

    return(A)


def mpsTomography(sites, observables, expectations):
    """
    Description:
        Construct an MPS and optimize the tensors to match the target expectations.
    Args:
        sites (int): The number of sites in the MPS.
        observables (list): A list of observables to optimize over.
        expectations (list): A list of the target expectations.
    Returns:
        Ms (list): A list of the optimized MPS tensors.
        Ss (list): A list of the optimized singular values.
    """

    # Initilize random MPS state. np.random.rand populates the tensor with random numbers between 0 and 1.
    Ls = [np.random.rand(1, 2, 2)]
    Ls += [np.random.rand(2, 2, 2) for i in range(sites-2)]
    Ls += [np.random.rand(2, 2, 1)]
    Ss = [np.random.rand(2) for i in range(sites-1)]

    # Sweep over the sites and optimize based on the list of observables and associated expectations.
    # TO DO: Fix some reshaping issues. This should be implemented similar to the mpsDecomp function.
    for i in range(sites-1):
        if i == 0:
            A = np.einsum('db, abc -> adc', np.diag([1,1]), Ls[0])
            A = np.einsum('abc, cd -> abd', A, np.diag(Ss[0]))
            A = np.einsum('abc, dcf -> adbf', A, Ls[1])
            A = np.einsum('abcd, de -> abce', A, np.diag(Ss[1]))
            A = optimizeSite(A, observables[i], expectations[i])
            A = np.reshape(A, (2, 4))
            U, Ss[0], V = np.linalg.svd(A)
            Ls[0] = np.reshape(U, (1,2,2))
            A = np.reshape(V[:2], (2,2,2))
        elif i == sites-2:
            A = np.einsum('db, abc -> adc', np.diag(Ss[i-1]), A)
            A = np.einsum('abc, dcf -> adbf', A, Ls[i+1])
            # A = np.einsum('abcd, de -> abce', A, np.diag(Ss[i+1]))
            A = optimizeSite(A, observables[i], expectations[i])
            # Ls[i] = np.linalg.pinv(np.diag(Ss[i-1])) @ Ls[i]
            A = np.reshape(A, (4, 2))
            U, Ss[-1], V = np.linalg.svd(A)
            Ls[-2] = np.reshape(U[:2], (2,2,2))
            Ls[-1] = np.reshape(V, (2,2,1))
            
        else:
            A = np.einsum('db, abc -> adc', np.diag(Ss[i-1]), A)
            A = np.einsum('abc, dcf -> adbf', A, Ls[i+1])
            A = np.einsum('abcd, de -> abce', A, np.diag(Ss[i+1]))
            A = optimizeSite(A, observables[i], expectations[i])
            # Ls[i] = np.linalg.pinv(np.diag(Ss[i-1])) @ Ls[i]
    # Return optimized MPS.

    return (Ls, Ss)

sites = state.num_qubits

observables = []
expectations = []
for i in range(sites-1):
    temp1 = []
    temp2 = []
    for pair in itertools.combinations_with_replacement(["X", "Y", "Z", "I"], 2):
        temp1.append(''.join(pair))
        temp2.append(state.expectation_value(Pauli(''.join(pair))))
    observables.append(temp1)
    expectations.append(temp2)


# Ls, Ss = mpsTomography(sites, observables, expectations)

# print(Ls, Ss)

# mpsSingleSiteExpectation(Ls, Ss, 'Z', 1)

In [651]:
import numpy as np
from itertools import product

def pauli_matrices():
    """Returns a dictionary of Pauli matrices."""
    I = np.array([[1, 0], [0, 1]], dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    return {'I': I, 'X': X, 'Y': Y, 'Z': Z}

def reconstruct_two_qubit_density_matrix(expectations):
    """
    Reconstructs a two-qubit density matrix from expectation values.

    Parameters:
        expectations (dict): Expectation values for {'II', 'IX', 'IY', ... 'ZZ'}.

    Returns:
        np.array: 4x4 density matrix r.
    """
    paulis = pauli_matrices()
    r = np.zeros((4, 4), dtype=complex)
    
    for i, Ps in enumerate(expectations[0]):
        P1, P2 = Ps[0], Ps[1]
        P = np.kron(paulis[P1], paulis[P2])  # Tensor product of Pauli operators
        r += expectations[1][i] * P

    print(np.linalg.eig(r/4))
    
    return r / 4  # Normalize

def estimate_mps(num_qubits, expect_two, max_bond_dim=2):
    """
    Estimates MPS tensors from expectation values.

    Parameters:
        expect_single (list): List of single-qubit expectation values.
        expect_two (list): List of two-qubit expectation values.
        max_bond_dim (int): Maximum bond dimension.

    Returns:
        list: MPS tensors.
    """
    mps_tensors = []

    for i in range(num_qubits - 1):
        r_ij = reconstruct_two_qubit_density_matrix(expect_two[i])

        # Reshape into a 2x2x2x2 tensor (2 for each qubit)
        r_ij = r_ij.reshape(2, 2, 2, 2)

        # Reshape for SVD (merge physical indices)
        r_ij = r_ij.transpose(0, 2, 1, 3).reshape(4, 4)
        
        # Singular Value Decomposition
        U, S, Vh = np.linalg.svd(r_ij, full_matrices=False)

        # Truncate bond dimension
        S = np.diag(S[:max_bond_dim])
        U = U[:, :max_bond_dim]
        Vh = Vh[:max_bond_dim, :]

        # Reshape MPS tensors
        U = U.reshape(2, 2, max_bond_dim)
        Vh = Vh.reshape(max_bond_dim, 2, 2)

        if i == 0:
            mps_tensors.append(U)  # First tensor

        mps_tensors.append(Vh)  # Next core tensor

    return mps_tensors


sites = 3
two_qubit_exp = []
for i in range(sites-1):
    temp1 = []
    temp2 = []
    for pair in itertools.combinations_with_replacement(["X", "Y", "Z", "I"], 2):
        temp1.append(''.join(pair))
        temp2.append(state.expectation_value(Pauli(''.join(pair))))

    two_qubit_exp.append((temp1,temp2))

print(two_qubit_exp)
mps_result = estimate_mps(sites, two_qubit_exp, max_bond_dim=2)

for i, tensor in enumerate(mps_result):
    print(f"MPS tensor {i}:\n", tensor.shape, "\n", tensor, "\n")


[(['XX', 'XY', 'XZ', 'XI', 'YY', 'YZ', 'YI', 'ZZ', 'ZI', 'II'], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999998, 0.0, 0.9999999999999999]), (['XX', 'XY', 'XZ', 'XI', 'YY', 'YZ', 'YI', 'ZZ', 'ZI', 'II'], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999998, 0.0, 0.9999999999999999])]
EigResult(eigenvalues=array([5.00000000e-01+0.j, 2.77555756e-17+0.j, 2.77555756e-17+0.j,
       5.00000000e-01+0.j]), eigenvectors=array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]]))
EigResult(eigenvalues=array([5.00000000e-01+0.j, 2.77555756e-17+0.j, 2.77555756e-17+0.j,
       5.00000000e-01+0.j]), eigenvectors=array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]]))
MPS tensor 0:
 (2, 2, 2) 
 [[[-0.70710678+0.j  0.70710678+0.j]
  [ 0.        +0.j  0.        +0.j]]

 [[ 0.      

In [652]:
def reconstruct_3qstate_vector(pauli_labels, expectation_values):
    """
    Reconstructs a pure state vector from expectation values of 2-qubit Pauli matrices.
    
    Parameters:
        expectation_values (list or np.array): Expectation values of 2-qubit Pauli matrices.
    
    Returns:
        np.array: The reconstructed state vector.
    """
    # Define single-qubit Pauli matrices
    pauli_matrices = {
        'I': np.eye(2),
        'X': np.array([[0, 1], [1, 0]]),
        'Y': np.array([[0, -1j], [1j, 0]]),
        'Z': np.array([[1, 0], [0, -1]])
    }
    
    # Generate all 2-qubit Pauli matrices
    # pauli_labels = [''.join(p) for p in product('IXYZ', repeat=2)]
    pauli_operators = [np.kron(np.kron(pauli_matrices[p[0]], pauli_matrices[p[1]]), pauli_matrices[p[2]]) for p in pauli_labels]
    
    # Reconstruct density matrix
    rho = (1/8) * sum(e * P for e, P in zip(expectation_values, pauli_operators))
    
    # Compute eigenvalues and eigenvectors
    eigvals, eigvecs = np.linalg.eigh(rho)

    state = [0 for _ in range(8)]
    for i, e in enumerate(eigvals):
        if abs(e) > 1e-10:
            state += np.sqrt(e)*eigvecs[:, i]
    
    state = state/np.linalg.norm(state)
    
    return Statevector(state)


def generate_mps(sites, pauli_labels, expectations, bond_dim):
    Ls = []
    Ss = []
    Us = []

    if sites == 3:
        return mpsDecomp(state, bond_dim)

    for n in range(sites-2):
        localPsi = reconstruct_3qstate_vector(pauli_labels[n], expectations[n])
        L, S, U = mpsDecomp(localPsi, bond_dim)
        Ss.append(S[0])
        if n == 0:
            Ls.append(L[0])
            Ls.append(L[1])
            Us.append(U[0])
            Us.append(U[1])
        elif n == sites-3:
            Ls.append(L[1])
            Ls.append(L[2])
            Us.append(U[1])
            Us.append(U[2])
            Ss.append(S[1])
        else:
            Ls.append(L[1])
            Us.append(U[1])
            


    return (Ls, Ss, Us)


sites = state.num_qubits
measurementStats = []
pauli_labels = [[''.join(p) for p in product('IXYZ', repeat=3)] for _ in range(sites-2)]
for i in range(sites-2):
    expectations = []
    for p in pauli_labels[0]:
        expectations.append(state.expectation_value(Pauli(p), [j for j in range(sites-3, sites)]))

    measurementStats.append(expectations)

res = generate_mps(sites, pauli_labels, measurementStats, 2)

mps_to_state_vec(res[0], res[1]).draw("latex")

<IPython.core.display.Latex object>

# Benchmarks
Apply some distance measure to the MPS states generated through the MPS tomography and the true MPS state. Best option is to compute their inner product and report a precent differene from 1. Additionally, give accuracy comparisons between a shadow tomography aided approach and a pure MPS tomography approach. These should be plotted aginst the number of samples/copies of the quantum state that would be required to theoreticall obtain all the of the relavent measurement statistics.

## MPS Inner Product

In [653]:
Ls = [np.array([[[1.+0.j, 0.+0.j]],
        [[0.+0.j, 1.+0.j]]]), np.array([[[ 1.41421356+0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j]],

       [[ 0.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -1.41421356+0.j]]]), np.array([[[ 1.+0.j],
        [ 0.+0.j]],

       [[ 0.+0.j],
        [-1.+0.j]]])] 

Ss = [np.array([0.70710678, 0.70710678]), np.array([0.70710678, 0.70710678])] 

Us = [np.array([[[1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]]), np.array([[[ 1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j]],

       [[ 0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j]]]), np.array([[[ 0.70710678+0.j],
        [ 0.        +0.j]],

       [[ 0.        +0.j],
        [-0.70710678+0.j]]])]

In [654]:
# Flatten and reshape process.
print(Us[1])
s = Us[1].shape
x = Us[1].flatten()
print(x)
print(np.reshape(x, s))

[[[ 1.+0.j  0.+0.j]
  [ 0.+0.j  0.+0.j]]

 [[ 0.+0.j  0.+0.j]
  [ 0.+0.j -1.+0.j]]]
[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]
[[[ 1.+0.j  0.+0.j]
  [ 0.+0.j  0.+0.j]]

 [[ 0.+0.j  0.+0.j]
  [ 0.+0.j -1.+0.j]]]


In [655]:
def InnerMPS(Us1, Us2):
    """
    Description:
        Calculate the inner product of two MPS states.
    Args:
        Us1 (list): The first MPS state.
        Us2 (list): The second MPS state.
    Returns:
        inner (float): The inner product of the two MPS states.
    """
    
    for i in range(len(Us1)):
        if i == 0:
            res = np.einsum('ijk, ijk -> jk', Us1[i], Us2[i])
        elif i == len(Us1)-1:
            res = np.einsum('jk, ijk, ijk', res, Us1[i], Us2[i])
        else:
            res = np.einsum('jk, ijk, ijk -> jk', res, Us1[i], Us2[i])

    return res
        

print(InnerMPS(mpsDecomp(state, 2)[2], res[2]))

state.inner(mps_to_state_vec(res[0], res[1]))

(0.9999999999999998+0j)


(0.9999999999999998+0j)

In [656]:
def reconstruct_state(measurements, measurement_schemes):
    """
    Adapted from https://pennylane.ai/qml/demos/tutorial_classical_shadows/
    Given a classical shadow, this function reconstructs an approximation of the density matrix of the quantum state.
    
    Args:
        measurements (list): The classical shadows of the state.
        measurement_schemes (list): The measurement schemes.

    Returns:
        np.array: An approximation of the density matrix of the quantum state.
    """

    # Check if the number of qubits in the measurement schemes and the measurement results are the same.
    if measurement_schemes.shape != measurements.shape:
        raise ValueError("Mismatch in the number of qubits or the size in the measurement schemes and the measurement results.")

    # Get the number of qubits and the number of measurement schemes.
    size, num_qubits = measurement_schemes.shape

    # Computational basis states.
    zero_state = np.matrix([[1,0],[0,0]], dtype=complex)
    one_state = np.matrix([[0,0],[0,1]], dtype=complex)

    # Local qubit unitaries. 
    phase_z = np.matrix([[1,0],[0,-1j]], dtype=complex)
    hadamard = np.matrix([[1,1],[1,-1]], dtype=complex)/np.sqrt(2)
    identity = np.matrix([[1,0],[0,1]], dtype=complex)

    # Rotations to measure each of the paulis.
    X = hadamard
    Y = hadamard @ phase_z
    Z = identity

    # Tomographically complete set of unitaries. (The pauli basis is tomographically complete.) 
    unitary_ensemble = {'X': X, 'Y': Y, 'Z': Z}

    # Array to store the estimated density matrix.
    rho_estimated = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)

    for i in range(size):

        # Array to store the density matrix at each snapshot.
        rho_snapshot = [1]

        for j in range(num_qubits):

            # Implementation of formula S44 from the preskill paper.
            U = unitary_ensemble[measurement_schemes[i][j]]
            if measurements[i][j] == 1:
                b_state = zero_state
            else:
                b_state = one_state

            rho_local = 3 * (U.H @ b_state @ U) - identity

            rho_snapshot = np.kron(rho_snapshot, rho_local)

        # Average all the inverted basis states to get an approximation of the density matrix.
        rho_estimated += rho_snapshot/size

    return rho_estimated

def reconstruct_3qstate_vector(measurement_schemes, measurements):
    """
    Reconstructs a pure state vector from expectation values of 2-qubit Pauli matrices.
    
    Parameters:
        expectation_values (list or np.array): Expectation values of 2-qubit Pauli matrices.
    
    Returns:
        np.array: The reconstructed state vector.
    """
    # Define single-qubit Pauli matrices
    pauli_matrices = {
        'I': np.eye(2),
        'X': np.array([[0, 1], [1, 0]]),
        'Y': np.array([[0, -1j], [1j, 0]]),
        'Z': np.array([[1, 0], [0, -1]])
    }
    
    # Reconstruct density matrix
    rho = reconstruct_state(measurements, measurement_schemes)
    
    # Compute eigenvalues and eigenvectors
    eigvals, eigvecs = np.linalg.eigh(rho)

    state = [0 for _ in range(8)]
    for i, e in enumerate(eigvals):
        if e > 1e-10:
            state += np.sqrt(e)*eigvecs[:, i]
    
    state = state/np.linalg.norm(state)
    
    return Statevector(state)


def generate_mps(sites, measurement_schemes, measurements, bond_dim):
    Ls = []
    Ss = []
    Us = []

    if sites == 3:
        return mpsDecomp(state, bond_dim)

    for n in range(sites-2):
        localPsi = reconstruct_3qstate_vector(measurement_schemes[:, n:(n+3)], measurements[:, n:(n+3)])
        L, S, U = mpsDecomp(localPsi, bond_dim)
        Ss.append(S[0])
        if n == 0:
            Ls.append(L[0])
            Ls.append(L[1])
            Us.append(U[0])
            Us.append(U[1])
        elif n == sites-3:
            Ls.append(L[1])
            Ls.append(L[2])
            Us.append(U[1])
            Us.append(U[2])
            Ss.append(S[1])
        else:
            Ls.append(L[1])
            Us.append(U[1])
            


    return (Ls, Ss, Us)

## Test Case
For each test I will consider the following 3 test cases on 2-5 qubits. 
- GHZ State: $$ \ket{GHZ}_n = \frac{\ket{0}^{\otimes n} + \ket{1}^{\otimes n}}{\sqrt{2}}$$
- W State: $$ \ket{W}_n = \frac{1}{\sqrt{n}} \underbrace{(\ket{100..0} + \ket{010..0} + \cdots + \ket{000..1})}_{n}$$
- Cluster State: https://en.wikipedia.org/wiki/Cluster_state

## Test 1
Generate the MPS using the exact expectation values of the 3 pauli observables on each site.

In [657]:
test1 = Statevector.from_label("00000") + Statevector.from_label("11111")
test1 = test1/np.sqrt(test1.inner(test1))

trueMPS = mpsDecomp(test1, 2)

# Generate the measurement schemes.
n = test1.num_qubits
measurement_schemes = generate_pauli_measurement_schemes(n, 100000)

# Generate the classical shadow.
measurements = generate_classical_shadow(test1, measurement_schemes)

sites = test1.num_qubits

res = generate_mps(sites, measurement_schemes, measurements, 2)

mps_to_state_vec(res[0], res[1]).draw("latex")

# pauli_labels = [[''.join(p) for p in product('IXYZ', repeat=3)] for _ in range(sites-2)]
# for i in range(sites-2):
#     expectations = []
#     for p in pauli_labels[0]:
#         expectations.append(state.expectation_value(Pauli(p), [j for j in range(sites-3, sites)]))

KeyboardInterrupt: 

In [ ]:
t = test1.inner(mps_to_state_vec(res[0], res[1]))

print(t*np.conjugate(t))

(0.024865396816515797+0j)


## Test 2
Generate the MPS using the expectation values of the 3 pauli observables on each site calculated from a collection of measurements.

## Test 3
Generate the MPS using the expectation values of the 3 pauli observables on each site as estimated through a shadow tomography protocol.